<a href="https://colab.research.google.com/github/ElCald/CIFAR10/blob/main/TP2/2-how_to_use_lstm_for_one_to_many_many_to_one_and_many_to_many_sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemples simples de LSTM pour des séquences

# Imports and Setups

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from keras.models import Sequential

import numpy as np
import matplotlib.pyplot as plt

# Séquence *One to Many*

## Génération des données

In [ ]:
X, Y = [], []
X = [x+3 for x in range(-2, 43, 3)]

for i in X:
    output_vector = []
    output_vector.append(i+1)
    output_vector.append(i+2)
    Y.append(output_vector)

print(X)
print(Y)

Reformater l'entrée pour avoir la forme `[batch, timesteps, feature]`.

In [ ]:
X = np.array(X).reshape(15, 1, 1)
Y = np.array(Y)

print(f"Shape of X: {X.shape} and shape of Y: {Y.shape}")

## Créer un modèle

In [ ]:
def get_model():
    inputs = layers.Input(shape=(1,1))
    lstm = layers.LSTM(50, activation="relu")(inputs)
    outputs = layers.Dense(2)(lstm)

    model = models.Model(inputs, outputs)

    model.compile(optimizer='adam', loss='mse')

    return model

tf.keras.backend.clear_session()
model = get_model()
model.summary()

## Train

In [ ]:
history = model.fit(X, Y, epochs=50, validation_split=0.2, batch_size=3)


##History

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Prédiction

ici on passe la valeur 10. Le résultat devrait être un array [11, 12], ou plutôt quelque chose de proche ;)

In [ ]:
test_input = np.array([10]) # on s'attend à [11, 12] en sortie
test_input = test_input.reshape((1, 1, 1))
test_output = model.predict(test_input)
print(test_output)

##Est-ce qu'on peut faire mieux avec deux LSTM (Stacked LSTM) ?

Comme dans le cas des FCN et CNN, l'ajout d'une couche supplémentaire peut parfois augmenter la précision des résultats.

In [ ]:
def get_model():
    inputs = layers.Input(shape=(1,1))
    lstm1 = layers.LSTM(50, activation="relu",return_sequences=True)(inputs)
    lstm2 = layers.LSTM(25, activation="relu")(lstm1)
    outputs = layers.Dense(2)(lstm2)

    model = models.Model(inputs, outputs)

    model.compile(optimizer='adam', loss='mse')

    return model

tf.keras.backend.clear_session()

model = get_model()

model.summary()


## Train

In [ ]:
history = model.fit(X, Y, epochs=200, validation_split=0.2, verbose=1)


##History

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Prediction

In [ ]:
test_input = np.array([10]) # on s'attend à [11, 12] en sortie
test_input = test_input.reshape((1, 1, 1))
test_output = model.predict(test_input)
print(test_output)

# LSTM Bidirectionnel

Dans le fonctionnement "courant" d'un RNN (LSTM, GRU), la dépendance des données se fait dans un seul sens *chronologique*. Cependant, certaines données ont des dépendances dans les deux directions, alors on peut faire usage d'un LSTM Bidirectionnel. Dans notre exemple présenté ici,

In [ ]:


model = Sequential()
model.add(layers.Bidirectional(layers.LSTM(50, activation='relu'), input_shape=(1, 1)))
model.add(layers.Dense(2))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=100, validation_split=0.2, verbose=1, batch_size=3)


##History

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Prediction

In [ ]:
test_input = np.array([10]) # on s'attend à [11, 12] en sortie
test_input = test_input.reshape((1, 1, 1))
test_output = model.predict(test_input)
print(test_output)

Ok, toujours pas de [11, 12]... Mais avec un loss d'environ 2, il faudra s'attendre à une variation importante.

Dans ce cas, il faut essayer plus d'epochs et aussi de sauvegarder le meilleur modèle

# Many to One Sequence

Après le "One to many", maintenant on fait une séquence Many-to-One. Un exemple d'usage réel serait celui d'une prévision de la température à partir de l'humidité et de la pression atmosphérique. Dans l'exemple simple ci-dessous, on fait des séquences [1,2,3] et on essayer de prédire la somme [6].

## Data

In [ ]:
X = np.array([x+1 for x in range(45)])
X = X.reshape(15,3,1)

Y = []
for x in X:
    Y.append(x.sum())
Y = np.array(Y)

print(X)
print(Y)

## Model

In [ ]:
def get_model():
    inputs = layers.Input(shape=(3,1))
    lstm = layers.LSTM(50, activation="relu")(inputs)
    outputs = layers.Dense(1)(lstm)

    model = models.Model(inputs, outputs)

    model.compile(optimizer='adam', loss='mse')

    return model

tf.keras.backend.clear_session()
model = get_model()
model.summary()

## Train

In [ ]:
history = model.fit(X, Y, epochs=200, validation_split=0.2, verbose=0)


##History

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Prediction

In [ ]:
test_input = np.array([50,51,52]) ## on veut quelque chose proche de 153 en sortie (la somme des entrées)
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

# Exercice :
Essayez d'implémenter ce modèle en Pytorch.
* utiliser Dataset et Dataloader
* utiliser nn.LSTM

Quelles sont les similarités et différences que vous trouvez ? Pour un même nombre d'epochs, quelle implémentation (keras-tf ou pytorch) semble plus précise ?

**Déposer votre code python** (fichier .py avec l'ensemble de l'implémentation Pytorch) sur Moodle.

# Many to Many Sequence

Enfin, on peut faire du Many to Many, comme par exemple prédire la température des trois prochains jours à partir des trois derniers jours. Dans cet exemple, on a 3 valeurs [5, 10, 15] et on essaye de prédire la suite [20, 25, 30].

## Data

In [ ]:
X = list()
Y = list()
X = [x for x in range(5, 301, 5)]
Y = [y for y in range(20, 316, 5)]

X = np.array(X).reshape(20, 3, 1)
Y = np.array(Y).reshape(20, 3, 1)

print(X)
print(Y)

## Model

Dans ce modèle, on reçoit 3 entrées et on veut créer 3 sorties. Comme LSTM produit une seule sortie, nous allons *multiplier* les sorties en récupérant les étapes intermédiaures (`return_sequences = True`).

**Attention :** ceci ne marche car on a 3 features en entrée et 3 en sortie. Si le nombre ne correspond pas, il faudra adapter les tensors.

In [ ]:
def get_model():
    inputs = layers.Input(shape=(3, 1))  # sequence length 3, 1 feature
    lstm_out = layers.LSTM(100, activation='relu', return_sequences=True)(inputs)
    outputs = layers.Dense(1)(lstm_out)  # Dense applies to last dimension

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

tf.keras.backend.clear_session()
model = get_model()
model.summary()

## Train

In [ ]:
history = model.fit(X, Y, epochs=200, validation_split=0.2, verbose=0, batch_size=3)


##History

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Prediction

In [ ]:
test_input = np.array([300, 305, 310]) # on s'atten à avoir quelque chose comme [315, 320, 325] en sortie
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)